# llm4triton 项目演示

这个notebook展示了llm4triton项目的核心功能：一个标准化的评估框架，用于测试和优化PyTorch模型到Triton的性能。


## 1. 环境设置


In [1]:
import sys
import os
from pathlib import Path
import json
import torch

# 获取项目根目录
ROOT = Path.cwd().parent

# 切换到项目根目录
os.chdir(ROOT)

# 添加项目根目录到路径
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from harness.cases import make_case_forward, load_case_manifest
from harness.runner import execute_variant
from harness.data import seed_rng

print(f"项目根目录: {ROOT}")
print(f"当前工作目录: {Path.cwd()}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU设备: {torch.cuda.get_device_name(0)}")


项目根目录: /home/fanjingluo/explore/llm4triton
当前工作目录: /home/fanjingluo/explore/llm4triton
CUDA可用: True
GPU设备: NVIDIA A800 80GB PCIe


## 2. 加载测试案例

项目使用manifest.yaml来定义测试案例，让我们加载example_linear案例：


In [2]:
# 加载案例配置
manifest = load_case_manifest("example_linear")

print(f"案例名称: {manifest.display_name}")
print(f"描述: {manifest.description}")
print(f"模型类: {manifest.model_class}")
print(f"目标设备: {manifest.device}")
print(f"\n输入配置:")
print(json.dumps(manifest.inputs, indent=2))


案例名称: example_linear
描述: Two-layer MLP forward pass with GELU activation.
模型类: FeedForward
目标设备: cuda

输入配置:
{
  "args": [
    {
      "name": "x",
      "shape": [
        16,
        1024
      ],
      "dtype": "float32",
      "distribution": "normal",
      "mean": 0.0,
      "std": 1.0
    }
  ],
  "kwargs": {}
}


## 3. 查看模型结构

让我们实例化模型并查看其结构：


In [3]:
from cases.example_linear.model import FeedForward

# 创建模型实例
model = FeedForward()
print(model)
print(f"\n参数数量: {sum(p.numel() for p in model.parameters()):,}")


FeedForward(
  (lin1): Linear(in_features=1024, out_features=4096, bias=True)
  (act): GELU(approximate='none')
  (lin2): Linear(in_features=4096, out_features=1024, bias=True)
)

参数数量: 8,393,728


## 4. 数据缓存机制演示

项目使用确定性的数据缓存来确保baseline和优化版本使用完全相同的输入：


In [4]:
# 设置随机种子
seed_used = seed_rng(42)
print(f"使用种子: {seed_used}")

# 创建forward函数（这会自动处理数据缓存）
device = "cuda" if torch.cuda.is_available() else "cpu"
forward = make_case_forward("example_linear", device=device)

# 执行一次forward，查看缓存位置
output = forward()
print(f"\n输出形状: {output.shape}")
print(f"输出设备: {output.device}")

# 检查缓存文件
cache_dir = ROOT / "artifacts" / "_shared_cache" / "example_linear"
print(f"\n缓存目录: {cache_dir}")
if cache_dir.exists():
    print("缓存文件:")
    for f in cache_dir.iterdir():
        size_kb = f.stat().st_size / 1024
        print(f"  - {f.name}: {size_kb:.2f} KB")


使用种子: 42


/home/fanjingluo/explore/llm4triton/harness/data.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  snapshot = torch.load(cache_file, map_location="cpu")



输出形状: torch.Size([16, 1024])
输出设备: cuda:0

缓存目录: /home/fanjingluo/explore/llm4triton/artifacts/_shared_cache/example_linear
缓存文件:
  - inputs.pt: 65.62 KB
  - weights.pt: 32790.37 KB


## 5. 运行Baseline性能测试

使用评估框架运行baseline测试：


In [5]:
# 运行baseline测试
metrics_baseline = execute_variant(
    label="baseline",
    forward_impl=forward,
    warmup_iters=5,
    measure_iters=20,
    use_profiler=False
)

print("\n=== Baseline性能指标 ===")
print(f"延迟: {metrics_baseline['latency_ms']:.4f} ms")
print(f"吞吐量: {metrics_baseline['throughput_per_s']:.2f} 次/秒")
if metrics_baseline['max_memory_bytes']:
    memory_mb = metrics_baseline['max_memory_bytes'] / (1024 * 1024)
    print(f"峰值显存: {memory_mb:.2f} MB")
print(f"正确性: {metrics_baseline['correctness']}")
print(f"设备: {metrics_baseline['device_type']}")



=== Baseline性能指标 ===
延迟: 59.3219 ms
吞吐量: 16.86 次/秒
峰值显存: 106.25 MB
正确性: True
设备: cuda


## 6. 正确性验证演示

运行一个"candidate"变体，框架会自动与baseline对比验证正确性：


In [6]:
# 运行candidate测试（使用相同的forward实现作为演示）
metrics_candidate = execute_variant(
    label="candidate",
    forward_impl=forward,
    warmup_iters=5,
    measure_iters=20,
    baseline_label="baseline",
    compare_outputs=True
)

print("\n=== Candidate性能指标 ===")
print(f"延迟: {metrics_candidate['latency_ms']:.4f} ms")
print(f"吞吐量: {metrics_candidate['throughput_per_s']:.2f} 次/秒")
print(f"正确性验证: {'✓ 通过' if metrics_candidate['correctness'] else '✗ 失败'}")
if 'correctness_max_abs_diff' in metrics_candidate:
    print(f"最大绝对误差: {metrics_candidate['correctness_max_abs_diff']:.2e}")
    print(f"最大相对误差: {metrics_candidate['correctness_max_rel_diff']:.2e}")



=== Candidate性能指标 ===
延迟: 58.5953 ms
吞吐量: 17.07 次/秒
正确性验证: ✓ 通过
最大绝对误差: 0.00e+00
最大相对误差: 0.00e+00


/home/fanjingluo/explore/llm4triton/harness/runner.py:203: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expected_snapshot = torch.load(reference_path, map_location="cpu")


## 7. 性能对比分析


In [7]:
# 性能对比
speedup = metrics_baseline['latency_ms'] / metrics_candidate['latency_ms']

print("\n=== 性能对比 ===")
print(f"Baseline延迟:   {metrics_baseline['latency_ms']:.4f} ms")
print(f"Candidate延迟:  {metrics_candidate['latency_ms']:.4f} ms")
print(f"加速比:         {speedup:.2f}x")
print(f"\n正确性:        {metrics_candidate['correctness']}")



=== 性能对比 ===
Baseline延迟:   59.3219 ms
Candidate延迟:  58.5953 ms
加速比:         1.01x

正确性:        True


## 8. 查看保存的指标文件

所有指标都会自动保存到artifacts目录：


In [8]:
# 查看保存的metrics
baseline_metrics_path = ROOT / "artifacts" / "baseline" / "metrics.json"
candidate_metrics_path = ROOT / "artifacts" / "candidate" / "metrics.json"

print("保存的文件结构:")
artifacts_dir = ROOT / "artifacts"
for variant_dir in ["baseline", "candidate"]:
    var_path = artifacts_dir / variant_dir
    if var_path.exists():
        print(f"\n{variant_dir}/")
        for f in var_path.rglob('*'):
            if f.is_file():
                rel_path = f.relative_to(var_path)
                size_kb = f.stat().st_size / 1024
                print(f"  - {rel_path}: {size_kb:.2f} KB")


保存的文件结构:

baseline/
  - metrics.json: 0.31 KB
  - reference_outputs.pt: 65.22 KB

candidate/
  - metrics.json: 0.31 KB


## 9. Profiler演示（可选）

启用profiler来获取详细的性能分析：


In [9]:
# 使用profiler运行（仅在CUDA可用时）
if torch.cuda.is_available():
    print("运行带profiler的测试...")
    metrics_profiled = execute_variant(
        label="profiled_baseline",
        forward_impl=forward,
        warmup_iters=2,
        measure_iters=5,
        use_profiler=True
    )
    
    profiler_trace = ROOT / "artifacts" / "profiled_baseline" / "profiler" / "trace.json"
    if profiler_trace.exists():
        size_mb = profiler_trace.stat().st_size / (1024 * 1024)
        print(f"\nProfiler trace已保存到: {profiler_trace}")
        print(f"文件大小: {size_mb:.2f} MB")
        print("\n你可以在Chrome浏览器中打开 chrome://tracing 来查看此文件")
else:
    print("Profiler功能需要CUDA支持")


运行带profiler的测试...

Profiler trace已保存到: /home/fanjingluo/explore/llm4triton/artifacts/profiled_baseline/profiler/trace.json
文件大小: 0.13 MB

你可以在Chrome浏览器中打开 chrome://tracing 来查看此文件


## 总结

这个项目提供了以下核心功能：

1. **标准化评估框架**: 通过manifest.yaml定义测试案例，无需编写额外的测试代码
2. **确定性数据缓存**: 确保baseline和优化版本使用完全相同的输入数据，保证公平对比
3. **自动正确性验证**: 自动对比baseline和候选版本的输出，支持容差配置
4. **性能分析**: 测量延迟、吞吐量、显存使用等关键指标
5. **Profiler集成**: 支持PyTorch profiler，生成详细的性能trace
6. **结果归档**: 所有指标和trace自动保存到artifacts目录

下一步可以：
- 创建自己的测试案例（参考cases/TEMPLATE/）
- 实现Triton优化版本并对比性能
- 使用LangGraph集成的agent进行自动化优化
